Run the cell below to import the training and test data.

In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import optimizers, losses, callbacks

df1=pd.read_csv('/kaggle/input/nifty-50-stock-data-0115-to-0619/NIFTY 50-01-01-2015-to-31-12-2015.csv')
df2=pd.read_csv('/kaggle/input/nifty-50-stock-data-0115-to-0619/NIFTY 50-01-01-2016-to-31-12-2016.csv')
df3=pd.read_csv('/kaggle/input/nifty-50-stock-data-0115-to-0619/NIFTY 50-01-01-2017-to-31-12-2017.csv')
df4=pd.read_csv('/kaggle/input/nifty-50-stock-data-0115-to-0619/NIFTY 50-01-01-2018-to-31-12-2018.csv')
df5=pd.read_csv('/kaggle/input/nifty-50-stock-data-0115-to-0619/NIFTY 50-01-01-2019-to-30-06-2019.csv')

#training_data=pd.concat([df1, df2, df3, df4, df5])
training_data=pd.read_csv('/kaggle/input/stock-market-data-nifty-50-2000-2022/Nifty_50_with_indicators_.csv')
training_data['Date']=pd.to_datetime(training_data['Date'])
training_data = training_data.sort_values(by='Date')
#print(training_data)

# set the test data
test_data_input = pd.DataFrame({'open': training_data['open'].iloc[5581],
                                'high': training_data['high'].iloc[5581],
                                'low': training_data['low'].iloc[5581]}, index=[5581])

test_data_output = pd.DataFrame({'close': training_data['close'].iloc[5581]}, index=[5581])
#print(test_data_input)
#print(test_data_output)
training_data=training_data.drop(0)
#print(training_data)

# set the training data
train_data_output = pd.DataFrame(training_data['close'][0:2792])
train_data_input = pd.DataFrame(training_data[['open', 'high', 'low']][0:2792])
print(train_data_input)
print(train_data_output)

#set the validation data
val_data_output = pd.DataFrame(training_data['close'][2792:5582])
val_data_input = pd.DataFrame(training_data[['open', 'high', 'low']][2792:5582])
print(val_data_input)
print(val_data_output)

Run the cell below to train your model.

In [ ]:
model = keras.Sequential([
    #input layer
    keras.layers.Dense(3, activation='relu', input_shape=[3]),
    #keras.layers.Dropout(0.2),

    #hidden layers
    #keras.layers.Dense(4096, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    
    #output layer
    keras.layers.Dense(1, activation='linear')
])

# Define early stopping callback
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Set the laerning rate
learning_rate = 0.001

# Choose the optimizer
optimizer = optimizers.Adam(learning_rate=learning_rate)
#optimizer = optimizers.RMSprop(learning_rate=learning_rate)
#optimizer = optimizers.SGD(learning_rate=learning_rate)
#optimizer = optimizers.Adagrad(learning_rate=learning_rate)

# Choose the loss function
#loss=losses.mean_absolute_error
loss=losses.mean_squared_error
#loss=losses.binary_crossentropy

model.compile(optimizer=optimizer, loss=loss)

model.fit(train_data_input ,train_data_output, validation_data=(val_data_input ,val_data_output), epochs=30, batch_size=100,  callbacks=[early_stopping])

Run the cell below to test your model.

In [ ]:
print("Actual : ", test_data_output.values[0][0])
print("Predicted : ",model.predict(test_data_input)[0][0])